In [111]:
# CS-340 Client/Server Development
# Project Two
# Marco Aldana
# 02/20/2021
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

import base64
import AnimalMaintenance

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "@n!m@l$"
animal_maintenance = AnimalMaintenance.AnimalMaintenance(username, password)

#class read method must support return of cursor object
data_frame = pd.DataFrame.from_records(animal_maintenance.read({}))

###########################
# Dashboard Layout / View
###########################

app = JupyterDash("SNHU CS-340 Project Two")

image_filename = 'grazioso-salvare-logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

del data_frame['_id']

app.layout = html.Div([
    # Adds an anchor tag to include Grazioso salvare logo with hyperlink attached
    html.A([
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={'width': '200px', 'height': '150px'})],
        href="https://www.snhu.edu/"
    ),
    html.H5("Marco Aldana's CS-340 Project Two"),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        #interactive filtering options
        className = 'row',
        style={'display': 'flex'},
        children=[
             dcc.RadioItems(
                 id='filter_type',
                 options=[
                     {'label': 'Water Rescue', 'value': 'water_rescue'},
                     {'label': 'Moutain Rescue', 'value': 'mountain_rescue'},
                     {'label': 'Disaster Rescue', 'value': 'disaster_rescue'},
                     {'label': 'Reset', 'value': 'reset'}
                 ]
             )  
        ]),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {
                "name": i,
                "id": i,
                "deletable": False,
                "selectable": True
            } for i in data_frame.columns
        ],
        data= data_frame.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = True, #Enable row_selectable for interactive data table
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10
    ),
    html.Br(),
    html.Hr(),
    
    # This sets up the dashboard so that your chart and geolocation chart are side-by-side
    html.Div(
        className ='row',
        style={'display' : 'flex'},
        children=[
            dcc.Graph(id="pie-chart", className='col s12 m6'),
            html.Div(
                id='map-id',
                className='col s12 m6'
            )
        ])
])

#############################################
# Interaction between components / controller
#############################################

@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'columns')],
    [Input('filter_type', 'value')])

#Dynamic datatable filtering based on selection of radio button
def update_datatable(filter_type):
    if (filter_type == "water_rescue"):
        #Get base query of preferred breeds
        data_frame = pd.DataFrame.from_records(animal_maintenance.read({
            "breed": { "$in": [
                "Labrador Retriever Mix",
                "Chesapeake Bay Retriever",
                "Newfoundland"]}}))
        
        #filter data result based on Preferred Sex
        data_frame = data_frame.query("sex_upon_outcome == 'Intact Female'")
        
        #filter data result based on Training Age
        data_frame = data_frame.query('age_upon_outcome_in_weeks >= 26')
        data_frame = data_frame.query('age_upon_outcome_in_weeks <= 156')
        
    elif (filter_type == 'mountain_rescue'):
        #Get base query of preferred breeds
        data_frame = pd.DataFrame.from_records(animal_maintenance.read({ 
            "breed" : { "$in": [
                "German Shepherd",
                "Alaskan Malamute",
                "Old English Sheepdog",
                "Siberian Husky",
                "Rottweiler"]}}))
        
        #filter data result based on Preferred Sex
        data_frame = data_frame.query("sex_upon_outcome == 'Intact Male'")
        
        #filter data result based on Training Age
        data_frame = data_frame.query('age_upon_outcome_in_weeks >= 26')
        data_frame = data_frame.query('age_upon_outcome_in_weeks <= 156')
                        
    elif (filter_type == 'disaster_rescue'):
        #Get base query of preferred breeds
        data_frame = pd.DataFrame.from_records(animal_maintenance.read(
            { "breed" : {"$in": [
                "Doberman Pinscher",
                "German Shepard",
                "Golden Retriever",
                "Bloodhound",
                "Rottweiler"
            ]}}))
            
        #filter data result based on Preferred Sex
        data_frame = data_frame.query("sex_upon_outcome == 'Intact Male'")
        
        #filter data result based on Training Age
        data_frame = data_frame.query('age_upon_outcome_in_weeks >= 20')
        data_frame = data_frame.query('age_upon_outcome_in_weeks <= 300')
            
    #Reset the data filter
    elif (filter_type == 'reset'):
        #Reset to original data call
        data_frame = pd.DataFrame.from_records(animal_maintenance.read({}))
     
    del data_frame['_id']
    
    data = data_frame.to_dict('records')
    columns=[
        {"name": i, "id": i, "deletable": False, "selectable": True} for i in data_frame.columns
    ]
    
    return (data, columns)

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]
    
#Pie Chart
@app.callback(
    Output('pie-chart', "figure"),
    [Input('datatable-id', "derived_viewport_data")])
    
def generate_chart(viewData):
    
    figure= px.pie(data_frame, names='breed', title='Animal Breeds')
    return figure

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)

#Geolocation chart
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]


app
